In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

In [2]:
tf.__version__

'2.11.0'

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset
X = dataset.drop(['CustomerId','RowNumber','Surname','Exited'],axis=1).iloc[:,:].values
Y = dataset.iloc[:,-1].values

In [4]:
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [5]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [7]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
#model
ann = tf.keras.models.Sequential()

In [9]:
dataset.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [10]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) #first hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) #output layer

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) #compile

#TRaining the ANN
ann.fit(X_train, y_train, batch_size = 32, epochs = 100,verbose=0)

In [11]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 79ms/step
[[False]]


In [12]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(y_pred)


63/63 [==============================] - 0s 659us/step
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


In [13]:
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1522   73]
 [ 199  206]]


0.864